In [1]:
import os, tarfile
import re, glob
import pandas as pd
import numpy as np
import zipfile, gzip
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
os.chdir("/data/projects/DNABERT_data/Core_promoters/Non_core_promoter_regions")

In [3]:
file_extension = '.csv'
all_filenames = [i for i in glob.glob(f"*{file_extension}")]

In [4]:
all_filenames

['chr3_non_core_promoter.csv',
 'chr4_non_core_promoter.csv',
 'chr6_non_core_promoter.csv',
 'chr10_non_core_promoter.csv',
 'chr20_non_core_promoter.csv',
 'chrY_non_core_promoter.csv',
 'chrX_non_core_promoter.csv',
 'chr12_non_core_promoter.csv',
 'chr18_non_core_promoter.csv',
 'chr22_non_core_promoter.csv',
 'chr7_non_core_promoter.csv',
 'chr5_non_core_promoter.csv',
 'chr19_non_core_promoter.csv',
 'chr11_non_core_promoter.csv',
 'chr14_non_core_promoter.csv',
 'chr8_non_core_promoter.csv']

In [5]:
combined_csv_data = pd.concat([pd.read_csv(f) for f in all_filenames])
combined_csv_data.drop_duplicates().reset_index(drop=True)

,Unnamed: 0,0
0,0,NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...
1,1,ctaaccctaaccctaaccctaaccctaaccctaaccctaaccctca...
2,2,cctcaccctcaccctcactcaaccctaaccctcaccctaacctcca...
3,3,accctaaccctaaccctaacccctaacccctaaccctaaccctaac...
4,4,ccaaccccaccccaaccccaaccctaaccctaaccctaaccctaac...
...,...,...
27456001,2036110,aagtatgtggtcaattttggaataggtgtggtgctgaaaaaatgtg...
27456002,2036111,gttctgtagatgtctattaggtccacttggtgtagagctgagttaa...
27456003,2036112,tctgtctcactgatctgtctaatgttgacagtggggtgttaaagtc...
27456004,2036113,taagtctctttgtaggtcactcaggacttgctttatgaatctgggt...


In [6]:
combined_csv_data = combined_csv_data[combined_csv_data["0"].str.contains("N") == False]

In [7]:
combined_csv_data

,Unnamed: 0,0
1,1,ctaaccctaaccctaaccctaaccctaaccctaaccctaaccctca...
2,2,cctcaccctcaccctcactcaaccctaaccctcaccctaacctcca...
3,3,accctaaccctaaccctaacccctaacccctaaccctaaccctaac...
4,4,ccaaccccaccccaaccccaaccctaaccctaaccctaaccctaac...
5,5,taaccctaaccctaaccctaaccctaaccctaaccctaaccctaac...
...,...,...
2036108,2036108,aagttgttcagtttccatgttgttgagcggttttgagtgagtttct...
2036109,2036109,cactgtggtctgagagatagtttgttataatttctgctcttttaca...
2036110,2036110,aagtatgtggtcaattttggaataggtgtggtgctgaaaaaatgtg...
2036111,2036111,gttctgtagatgtctattaggtccacttggtgtagagctgagttaa...


In [8]:
measurer = np.vectorize(len)
measurer(combined_csv_data.values.astype(str)).max(axis=0)

array([ 7, 70])

In [9]:
def seq2kmer(seq, k):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space
    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers

In [10]:
df_neg=pd.DataFrame()
df_neg['Sequence'] = combined_csv_data['0'].str.upper().apply(seq2kmer, args=(6,))

In [11]:
df_neg

,Sequence
1,CTAACC TAACCC AACCCT ACCCTA CCCTAA CCTAAC CTAA...
2,CCTCAC CTCACC TCACCC CACCCT ACCCTC CCCTCA CCTC...
3,ACCCTA CCCTAA CCTAAC CTAACC TAACCC AACCCT ACCC...
4,CCAACC CAACCC AACCCC ACCCCA CCCCAC CCCACC CCAC...
5,TAACCC AACCCT ACCCTA CCCTAA CCTAAC CTAACC TAAC...
...,...
2036108,AAGTTG AGTTGT GTTGTT TTGTTC TGTTCA GTTCAG TTCA...
2036109,CACTGT ACTGTG CTGTGG TGTGGT GTGGTC TGGTCT GGTC...
2036110,AAGTAT AGTATG GTATGT TATGTG ATGTGG TGTGGT GTGG...
2036111,GTTCTG TTCTGT TCTGTA CTGTAG TGTAGA GTAGAT TAGA...


In [15]:
df_neg['Label']=0

In [16]:
df_neg = df_neg.reset_index(drop=True)
df_neg

,Sequence,Label
0,CTAACC TAACCC AACCCT ACCCTA CCCTAA CCTAAC CTAA...,0
1,CCTCAC CTCACC TCACCC CACCCT ACCCTC CCCTCA CCTC...,0
2,ACCCTA CCCTAA CCTAAC CTAACC TAACCC AACCCT ACCC...,0
3,CCAACC CAACCC AACCCC ACCCCA CCCCAC CCCACC CCAC...,0
4,TAACCC AACCCT ACCCTA CCCTAA CCTAAC CTAACC TAAC...,0
...,...,...
27455277,AAGTTG AGTTGT GTTGTT TTGTTC TGTTCA GTTCAG TTCA...,0
27455278,CACTGT ACTGTG CTGTGG TGTGGT GTGGTC TGGTCT GGTC...,0
27455279,AAGTAT AGTATG GTATGT TATGTG ATGTGG TGTGGT GTGG...,0
27455280,GTTCTG TTCTGT TCTGTA CTGTAG TGTAGA GTAGAT TAGA...,0


In [17]:
df_neg.to_csv("/data/projects/DNABERT_data/Core_promoters/negative_set.tsv", sep="\t", index=False)